## Ejercicio 2: Generar .py de funciones y main con al menos dos argumentos de entrada.

In [30]:
%%writefile funciones.py
import argparse
import subprocess
import time
import mlflow
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.datasets import load_wine
from pyngrok import ngrok

def argumentos():
    parser = argparse.ArgumentParser(description='__main__ de la aplicación con argumentos de entrada.')
    parser.add_argument('--nombre_job', type=str, help='Nombre del experimento en MLflow.')
    return parser.parse_args()

def load_dataset():
    wine = load_wine()
    df = pd.DataFrame(wine['data'], columns=wine['feature_names'])
    df['target'] = wine['target']
    return df

def data_treatment(df):
    train, test = train_test_split(df, test_size=0.2, random_state=42)
    test_target = test['target']
    test[['target']].to_csv('test-target.csv', index=False)
    del test['target']
    test.to_csv('test.csv', index=False)

    features = [x for x in list(train.columns) if x != 'target']
    x_raw = train[features]
    y_raw = train['target']

    X_train, X_test, y_train, y_test = train_test_split(x_raw, y_raw,
                                                        test_size=0.20,
                                                        random_state=123,
                                                        stratify=y_raw)
    return X_train, X_test, y_train, y_test

def setup_ngrok(port=6000):
    ngrok.set_auth_token('token_aqui') # indicar token
    ngrok.kill()
    public_url = ngrok.connect(port)
    print("ngrok URL:", public_url)
    return public_url

def mlflow_tracking(nombre_job, X_train, X_test, y_train, y_test, port=6000):
    mlflow_ui_process = subprocess.Popen(['mlflow', 'ui', '--port', str(port)])
    print(mlflow_ui_process)
    time.sleep(5)
    mlflow.set_experiment(nombre_job)

    models_params = {
        'LogisticRegression': (LogisticRegression(max_iter=1000), {
            "model__C": [0.1, 1, 10],
            "model__solver": ["liblinear", "lbfgs"]
        }),
        'KNeighborsClassifier': (KNeighborsClassifier(), {
            "model__n_neighbors": [3, 5, 7],
            "model__weights": ["uniform", "distance"]
        })
    }

    for model_name, (model, params) in models_params.items():
        with mlflow.start_run(run_name=model_name) as run:
            preprocessor = Pipeline(steps=[('scaler', StandardScaler())])
            clf = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('model', model)])
            
            grid_search = GridSearchCV(clf, params, cv=5, scoring='accuracy')
            grid_search.fit(X_train, y_train)

            best_params = grid_search.best_params_
            accuracy_train = grid_search.score(X_train, y_train)
            accuracy_test = grid_search.score(X_test, y_test)
            y_pred = grid_search.predict(X_test)
            report = classification_report(y_test, y_pred, output_dict=True)

            mlflow.log_params(best_params)
            mlflow.log_metric('accuracy_train', accuracy_train)
            mlflow.log_metric('accuracy_test', accuracy_test)
            mlflow.sklearn.log_model(grid_search.best_estimator_, model_name)

            report_df = pd.DataFrame(report).transpose()
            report_path = f"{model_name}_classification_report.csv"
            report_df.to_csv(report_path, index=True)
            mlflow.log_artifact(report_path)
    
    print("Se ha acabado el entrenamiento de los modelos correctamente")


Overwriting funciones.py


In [31]:
%%writefile main.py
from funciones import argumentos, load_dataset, data_treatment, setup_ngrok, mlflow_tracking

def main():
    print("Ejecutamos el main")
    args_values = argumentos()
    df = load_dataset()
    X_train, X_test, y_train, y_test = data_treatment(df)
    public_url = setup_ngrok(6000)
    mlflow_tracking(args_values.nombre_job, X_train, X_test, y_train, y_test, 6000)
    input("Presiona Enter para finalizar la sesión de ngrok...")
    ngrok.disconnect(public_url)

if __name__ == "__main__":
    main()


Overwriting main.py


In [29]:
!python main.py --nombre_job "ComparacionModelos"



Ejecutamos el main
ngrok URL: NgrokTunnel: "https://a4c7-46-37-82-200.ngrok-free.app" -> "http://localhost:6000"
<Popen: returncode: None args: ['mlflow', 'ui', '--port', '6000']>
[2024-07-12 19:45:54 +0200] [3402] [INFO] Starting gunicorn 22.0.0
[2024-07-12 19:45:54 +0200] [3402] [INFO] Listening at: http://127.0.0.1:6000 (3402)
[2024-07-12 19:45:54 +0200] [3402] [INFO] Using worker: sync
[2024-07-12 19:45:54 +0200] [3403] [INFO] Booting worker with pid: 3403
[2024-07-12 19:45:54 +0200] [3405] [INFO] Booting worker with pid: 3405
[2024-07-12 19:45:54 +0200] [3406] [INFO] Booting worker with pid: 3406
[2024-07-12 19:45:54 +0200] [3407] [INFO] Booting worker with pid: 3407
/Users/sdiamar/opt/anaconda3/envs/Ower/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Se ha acabado el entrenamiento de los modelos correctamente
Presiona Enter para finalizar la sesión de ngrok...^C
Trac

Adjunto capturas de pantalla de MLFlow para verificar de nuevo que todo ha ido bien:

![Modelos inicio colab](./images_mlflow/Modelos_inicio_main.png).
![Modelos inicio colab](./images_mlflow/KNN_Overview_main.png).
![Modelos inicio colab](./images_mlflow/KNN_Metrics_main.png).
![Modelos inicio colab](./images_mlflow/KNN_Artifacts_main.png).
![Modelos inicio colab](./images_mlflow/KNN_Report_main.png).
![Modelos inicio colab](./images_mlflow/LR_Overview_main.png).
![Modelos inicio colab](./images_mlflow/LR_Metrics_main.png).
![Modelos inicio colab](./images_mlflow/LR_Artifacts_main.png).
![Modelos inicio colab](./images_mlflow/LR_Report_main.png).

## Ejercicio 3: Práctica Flask

He hecho esta parte de la práctica con Flask y lo he adjuntado al repositorio en un archivo llamado app.py y las carpetas pertinentes para su correcta ejecución y salida por ngrok. Debido a falta de tiempo no he podido desplegarlo en Google Cloud Run como paso opcional.
He desarrollado una web que contiene una página de inicio, dos páginas donde se ejecutan algoritmos de HF como análisis de sentimiento y generación de texto, y por último, he añadido dos módulos más para mostrar los resultados de los modelos ejecutados en el ejercicio 1: LR y KNN.

En este ejercicio, he obtenido el token de ngrok a través de un archivo .env con mi clave y cargándolo en el archivo con load_dotenv.